In [1]:
import os
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
import pandas as pd
import numpy as np
from tensorflow.keras.backend import sigmoid
import tensorflow as tf
import tensorflow.keras.backend as K
from multiprocessing.dummy import Pool as ThreadPool

def swish(x, beta = 1):
    # https://www.geeksforgeeks.org/ml-swish-function-by-google-in-keras/
    return (x * sigmoid(beta * x))
def leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.25)
def image_convert(a):
    new = np.zeros((30, 30))
    for i in range(30):
        if int(a[i]*100)==0:
            d = 29
        elif int(a[i]*100)==100:
            d = 0
        else:
            d = 30-int(30-(1-a[i])*30)
            if d==30:
                d = 29
        new[d, i] = 1
    return new

def df_parsing(data_all_, c_):
    data_predict = data_all_[[x for x in data_all_.columns.tolist() if c_ in x]]
    data_predict_ = data_predict.copy()
    data_predict_['日期'] = data_all_['日期']
    a = pd.melt(data_predict_, id_vars=['日期'], value_vars=[x for x in data_predict_.columns.tolist() if 'pr' not in x])
    b = pd.melt(data_predict_, id_vars=['日期'], value_vars=[x for x in data_predict_.columns.tolist() if 'pr' in x])
    a.columns = ['日期', 'var', 'before']
    b.columns = ['日期', 'var', 'after']
    b['var'] = b['var'].str.replace("-pr","")
    c = pd.merge(a,b, on = ["日期","var"] , how = "left")
    return c

get_custom_objects().update({'swish': swish})
get_custom_objects().update({'leaky_relu': leaky_relu})

In [2]:
banana_X_train = pd.read_pickle("veg_X_train.pkl")
banana_X_val = pd.read_pickle("veg_X_val.pkl")
banana_Y_train = pd.read_pickle("veg_Y_train.pkl")
banana_Y_val = pd.read_pickle("veg_Y_val.pkl")
cols_name = [[y+'-'+str(x).rjust(2,"0") for x in range(1,31)] for y in ['RSV','上價中位數','下價中位數','中價中位數','交易量','平均價','雨量']]
cols_name_2 = [x for x in banana_X_train.columns.tolist() if '-' not in x]

train_X_num = banana_X_train[cols_name_2[1:]].values
val_X_num = banana_X_val[cols_name_2[1:]].values

train_Y_1 = banana_Y_train[[x for x in banana_Y_train.columns if 'RSV' in x]].values
val_Y_1 = banana_Y_val[[x for x in banana_Y_val.columns if 'RSV' in x]].values

In [3]:
train_X_lstm_1 = []
for i in range(len(cols_name)):
    tmp = banana_X_train[cols_name[i]].iloc[:,-7:]
    train_X_lstm_1.append(tmp)
train_X_lstm_1 = np.stack(train_X_lstm_1,axis=2)
val_X_lstm_1 = []
for i in range(len(cols_name)):
    tmp = banana_X_val[cols_name[i]].iloc[:,-7:]
    val_X_lstm_1.append(tmp)
val_X_lstm_1 = np.stack(val_X_lstm_1,axis=2)

train_X_lstm_2 = []
for i in range(len(cols_name)):
    tmp_all = []
    tmp = banana_X_train[cols_name[i]].iloc[:,-13:]
    for j in range(7):
        tmp_1 = tmp.iloc[:,j:j+7]
        tmp_all.append(tmp_1)
    tmp_all = np.stack(tmp_all,axis=2)
    train_X_lstm_2.append(tmp_all)
train_X_lstm_2 = np.stack(train_X_lstm_2,axis=3)

val_X_lstm_2 = []
for i in range(len(cols_name)):
    tmp_all = []
    tmp = banana_X_val[cols_name[i]].iloc[:,-13:]
    for j in range(7):
        tmp_1 = tmp.iloc[:,j:j+7]
        tmp_all.append(tmp_1)
    tmp_all = np.stack(tmp_all,axis=2)
    val_X_lstm_2.append(tmp_all)
val_X_lstm_2 = np.stack(val_X_lstm_2,axis=3)

train_X_lstm_4 = np.stack([np.stack([image_convert(x) for x in banana_X_train[cols_name[y]].values]) for y in range(len(cols_name))], axis=3)
val_X_lstm_4 = np.stack([np.stack([image_convert(x) for x in banana_X_val[cols_name[y]].values]) for y in range(len(cols_name))], axis=3)

data_X = {'Conv1D':[train_X_lstm_1, val_X_lstm_1],
          'Vanilla':[train_X_lstm_1, val_X_lstm_1],
          'Stacked':[train_X_lstm_1, val_X_lstm_1],
          'Bidirectional':[train_X_lstm_1, val_X_lstm_1],
          'Conv1D_LSTM':[train_X_lstm_2, val_X_lstm_2],
          'Conv2D_1':[train_X_lstm_2, val_X_lstm_2],
          'Conv2D_2':[train_X_lstm_4, val_X_lstm_4]}

In [4]:
def model_predict(list_route_, model_, x1, x2, Y, dataset):
    #{'高於成本價格': 0, '低於成本價格': 1, '上漲': 0, '持平':1, '下跌':2}
    data_ = model_.predict([x1,x2], 
                           verbose = 0)#, use_multiprocessing = True, workers = 4, 
                           #max_queue_size = 20, batch_size = 256)
    data_ = data_[:,:,0]
    data_ = pd.DataFrame(data_)
    data_new = data_.copy()
    data_.columns = ['RSV-02-pr','RSV-03-pr','RSV-04-pr','RSV-05-pr','RSV-06-pr']
    data_new[(data_new>1)] = 1
    data_new[(data_new<0)] = 0
    data_new[data_new<=0.2] = 2
    data_new[(data_new>=0.8)&(data_new<=1)] = 0
    data_new[(data_new>0.2)&(data_new<0.8)] = 1
    data_new.columns = ['漲跌狀態-02-pr','漲跌狀態-03-pr','漲跌狀態-04-pr','漲跌狀態-05-pr','漲跌狀態-06-pr']
    data_ = pd.concat([data_, data_new] , axis = 1)
    data_all = Y.copy()
    for c_ in ['成本價格-02','成本價格-03','成本價格-04','成本價格-05','成本價格-06']:
        del data_all[c_]
    data_all = data_all.reset_index(drop = True)
    data_all = pd.concat([data_all, data_] , axis = 1)
    #data_RSV = df_parsing(data_all, c_ = "RSV")
    data_class = df_parsing(data_all, c_ = "漲跌狀態")
    data_class['result'] = ''
    data_class.loc[data_class['before']==data_class['after'], 'result'] = True
    data_class.loc[data_class['before']!=data_class['after'], 'result'] = False
    data_class_groupby = data_class.groupby(['var','before','result']).size().rename('num').reset_index(drop = False)
    data_class_groupby = data_class_groupby.pivot(index='var', columns=['before','result'], values='num')
    data_class_groupby.columns = [str(int(x[0]))+"-"+str(x[1]) for x in data_class_groupby.columns]
    data_class_groupby = data_class_groupby.reset_index()
    
    miss_col = list(set(['0-True', '1-True', '2-True', '0-False', '1-False', '2-False']) - set(data_class_groupby.columns.tolist()))
    if len(miss_col)>0:
        for c_ in miss_col:
            data_class_groupby[c_] = 0

    for c_ in ['0','1','2']:
        data_class_groupby[c_+"-per"] = data_class_groupby[c_+'-True']/(data_class_groupby[c_+'-True']+data_class_groupby[c_+'-False'])
    data_class_groupby = data_class_groupby.fillna(0)
    data_class_groupby['total-per'] = (data_class_groupby['0-True']+data_class_groupby['1-True']+data_class_groupby['2-True']) \
        /(data_class_groupby['0-True']+data_class_groupby['1-True']+data_class_groupby['2-True']+ \
          data_class_groupby['0-False']+data_class_groupby['1-False']+data_class_groupby['2-False'])
    data_class_groupby['source'] = list_route_
    data_all['source'] = list_route_
    data_class_groupby['dataset'] = dataset
    data_all['dataset'] = dataset
    
    evaluate_ = model_.evaluate([x1, x2], 
                                data_all[[x for x in data_all.columns if ('pr' not in x)&('RSV' in x)]].values,
                                verbose = 0)#, use_multiprocessing = True, workers = 4, 
                                #max_queue_size = 20, batch_size = 256)
    
    return data_class_groupby, data_all, {'source':list_route_, 'dataset':dataset, 'evaluate':evaluate_[1]}

def predict_all(list_route, data_X_, dataset):
    if dataset == "train":
        X = data_X_[list_route.split("/")[1]][0]
        Y = train_X_num
        al = banana_Y_train
    else:
        X = data_X_[list_route.split("/")[1]][1]
        Y = val_X_num
        al = banana_Y_val

    data_class_groupby = []
    data_all = []
    evaluate = []
    for h5 in ["final.h5", 'weights_loss.hdf5', 'weights_mse.hdf5']:
        list_route_ = list_route.replace("history.png", h5)
        final_model_ = load_model(list_route_)
        data_class_groupby_, data_all_, evaluate_ = model_predict(list_route_, final_model_, x1 = X, x2 = Y, Y = al, dataset = dataset)
        data_class_groupby.append(data_class_groupby_)
        data_all.append(data_all_)
        evaluate.append(evaluate_)
        del list_route_, final_model_, data_class_groupby_, data_all_, evaluate_
    data_class_groupby = pd.concat(data_class_groupby).reset_index(drop = True)
    data_all = pd.concat(data_all).reset_index(drop = True)
    return data_class_groupby, data_all, evaluate

def cal_all_acc(list_route_):
    data_all = []
    data_groupby = []
    data_evaluate = []
    data_error = []
    for i in range(len(list_route_)):
        print(i, len(list_route_))
        try:
            dataset_train = predict_all(list_route_[i], data_X, 'train')
            dataset_val = predict_all(list_route_[i], data_X, 'val')
            data_class_groupby_ = pd.concat([dataset_train[0], dataset_val[0]]).reset_index(drop = True)
            data_all_ = pd.concat([dataset_train[1], dataset_val[1]]).reset_index(drop = True)
            data_evaluate.append(dataset_train[2])
            data_evaluate.append(dataset_val[2])

            data_all.append(data_all_)
            data_groupby.append(data_class_groupby_)
            
            del dataset_train, dataset_val, data_class_groupby_, data_all_
            
        except:
            print("error", list_route_[i])
            data_error.append(list_route_[i])
    
    data_all = pd.concat(data_all).reset_index(drop = True)
    data_groupby = pd.concat(data_groupby).reset_index(drop = True)
    return data_all, data_groupby, data_evaluate, data_error

In [5]:
list_route = [os.path.join(path, name) for path, subdirs, files in os.walk('model') for name in files]
list_route = [x for x in list_route if 'history.png' in x]
print(len(list_route))
data = cal_all_acc(list_route)
data[0].to_csv("inference_all_veg_regression.csv", index = False)
data[1].to_csv("inference_groupby_veg_regression.csv", index = False)
pd.DataFrame(sum(data[2],[])).to_csv("inference_mse_veg_regression.csv", index = False)

1937
0 1937
1 1937
2 1937
3 1937
4 1937
5 1937
6 1937
7 1937
8 1937
9 1937
10 1937
11 1937
12 1937
13 1937
14 1937
15 1937
16 1937
17 1937
18 1937
19 1937
20 1937
21 1937
22 1937
23 1937
24 1937
25 1937
26 1937
27 1937
28 1937
29 1937
30 1937
31 1937
32 1937
33 1937
34 1937
35 1937
36 1937
37 1937
38 1937
39 1937
40 1937
41 1937
42 1937
43 1937
44 1937
45 1937
46 1937
47 1937
48 1937
49 1937
50 1937
51 1937
52 1937
53 1937
54 1937
55 1937
56 1937
57 1937
58 1937
59 1937
60 1937
61 1937
62 1937
63 1937
64 1937
65 1937
66 1937
67 1937
68 1937
69 1937
70 1937
71 1937
72 1937
73 1937
74 1937
75 1937
76 1937
77 1937
78 1937
79 1937
80 1937
81 1937
82 1937
83 1937
84 1937
85 1937
86 1937
87 1937
88 1937
89 1937
90 1937
91 1937
92 1937
93 1937
94 1937
95 1937
96 1937
97 1937
98 1937
99 1937
100 1937
101 1937
102 1937
103 1937
104 1937
105 1937
106 1937
107 1937
108 1937
109 1937
110 1937
111 1937
112 1937
113 1937
114 1937
115 1937
116 1937
117 1937
118 1937
119 1937
120 1937
121 1937
122 193

error model/Conv1D/0-adam-swish-leaky_relu/history.png
497 1937
error model/Conv1D/0.01-adam-tanh-leaky_relu/history.png
498 1937
error model/Conv1D/0.0001-nadam-gelu-leaky_relu/history.png
499 1937
error model/Conv1D/0.0001-adam-relu-gelu/history.png
500 1937
error model/Conv1D/0.0001-sgd-gelu-gelu/history.png
501 1937
error model/Conv1D/0-nadam-relu-swish/history.png
502 1937
error model/Conv1D/0.0001-sgd-relu-swish/history.png
503 1937
error model/Conv1D/0.01-adam-gelu-relu/history.png
504 1937
error model/Conv1D/0.1-adam-swish-gelu/history.png
505 1937
error model/Conv1D/0.1-adam-tanh-relu/history.png
506 1937
error model/Conv1D/0.001-sgd-swish-swish/history.png
507 1937
error model/Conv1D/0.01-sgd-tanh-swish/history.png
508 1937
error model/Conv1D/0.001-sgd-relu-leaky_relu/history.png
509 1937
error model/Conv1D/0.1-nadam-gelu-swish/history.png
510 1937
error model/Conv1D/0.1-sgd-tanh-leaky_relu/history.png
511 1937
error model/Conv1D/0.1-nadam-leaky_relu-swish/history.png
512 193

error model/Conv2D_2/0.001-sgd-leaky_relu-gelu/history.png
628 1937
error model/Conv2D_2/0.0001-nadam-swish-swish/history.png
629 1937
error model/Conv2D_2/0.001-nadam-swish-relu/history.png
630 1937
error model/Conv2D_2/0.1-sgd-gelu-relu/history.png
631 1937
error model/Conv2D_2/0.001-adam-relu-gelu/history.png
632 1937
error model/Conv2D_2/0.01-sgd-relu-relu/history.png
633 1937
error model/Conv2D_2/0.1-nadam-leaky_relu-gelu/history.png
634 1937
error model/Conv2D_2/0.001-adam-gelu-gelu/history.png
635 1937
error model/Conv2D_2/0.0001-sgd-relu-relu/history.png
636 1937
error model/Conv2D_2/0.0001-sgd-leaky_relu-swish/history.png
637 1937
error model/Conv2D_2/0.001-nadam-tanh-leaky_relu/history.png
638 1937
error model/Conv2D_2/0.1-adam-gelu-gelu/history.png
639 1937
error model/Conv2D_2/0-sgd-swish-gelu/history.png
640 1937
error model/Conv2D_2/0.001-nadam-tanh-swish/history.png
641 1937
error model/Conv2D_2/0.1-adam-relu-swish/history.png
642 1937
error model/Conv2D_2/0-nadam-leaky_

error model/Conv2D_2/0.0001-nadam-relu-leaky_relu/history.png
754 1937
error model/Conv2D_2/0.1-nadam-tanh-swish/history.png
755 1937
error model/Conv2D_2/0.0001-adam-leaky_relu-leaky_relu/history.png
756 1937
error model/Conv2D_2/0.01-sgd-relu-swish/history.png
757 1937
error model/Conv2D_2/0.0001-nadam-swish-gelu/history.png
758 1937
error model/Conv2D_2/0.1-adam-swish-leaky_relu/history.png
759 1937
error model/Conv2D_2/0.0001-nadam-swish-relu/history.png
760 1937
error model/Conv2D_2/0.01-adam-gelu-gelu/history.png
761 1937
error model/Conv2D_2/0.001-nadam-leaky_relu-gelu/history.png
762 1937
error model/Conv2D_2/0.001-sgd-gelu-leaky_relu/history.png
763 1937
error model/Conv2D_2/0.01-adam-tanh-swish/history.png
764 1937
error model/Conv2D_2/0-sgd-tanh-swish/history.png
765 1937
error model/Conv2D_2/0.001-nadam-relu-swish/history.png
766 1937
error model/Conv2D_2/0.001-sgd-swish-gelu/history.png
767 1937
error model/Conv2D_2/0.01-adam-gelu-leaky_relu/history.png
768 1937
error mode

error model/Conv2D_2/0.01-sgd-gelu-leaky_relu/history.png
881 1937
error model/Conv2D_2/0.1-nadam-swish-leaky_relu/history.png
882 1937
error model/Conv2D_2/0-adam-relu-relu/history.png
883 1937
error model/Conv2D_2/0.1-sgd-swish-leaky_relu/history.png
884 1937
error model/Conv2D_2/0-adam-swish-relu/history.png
885 1937
error model/Conv2D_2/0.1-nadam-swish-relu/history.png
886 1937
error model/Conv2D_2/0.0001-adam-swish-relu/history.png
887 1937
error model/Conv2D_2/0-sgd-swish-leaky_relu/history.png
888 1937
error model/Conv2D_2/0.1-adam-gelu-swish/history.png
889 1937
error model/Conv2D_2/0.1-adam-tanh-leaky_relu/history.png
890 1937
error model/Conv2D_2/0.001-sgd-tanh-relu/history.png
891 1937
error model/Conv2D_2/0.001-sgd-relu-relu/history.png
892 1937
error model/Conv2D_2/0.01-nadam-leaky_relu-gelu/history.png
893 1937
error model/Conv2D_2/0.001-adam-tanh-gelu/history.png
894 1937
error model/Conv2D_2/0.0001-adam-leaky_relu-swish/history.png
895 1937
error model/Conv2D_2/0-sgd-sw

...

error model/Bidirectional/0-nadam-swish-gelu/history.png
1365 1937
error model/Bidirectional/0.0001-nadam-leaky_relu-gelu/history.png
1366 1937
error model/Bidirectional/0.01-nadam-relu-leaky_relu/history.png
1367 1937
error model/Bidirectional/0.01-adam-leaky_relu-gelu/history.png
1368 1937
error model/Bidirectional/0.1-adam-leaky_relu-swish/history.png
1369 1937
error model/Bidirectional/0-adam-swish-gelu/history.png
1370 1937
error model/Bidirectional/0-sgd-tanh-gelu/history.png
1371 1937
error model/Bidirectional/0.1-adam-swish-swish/history.png
1372 1937
error model/Bidirectional/0.0001-adam-gelu-gelu/history.png
1373 1937
error model/Bidirectional/0.1-adam-gelu-relu/history.png
1374 1937
error model/Bidirectional/0.001-nadam-swish-swish/history.png
1375 1937
error model/Bidirectional/0.01-adam-swish-relu/history.png
1376 1937
error model/Bidirectional/0.001-nadam-gelu-gelu/history.png
1377 1937
error model/Bidirectional/0.1-adam-leaky_relu-leaky_relu/history.png
1378 1937
error m

error model/Conv1D_LSTM/0.001-adam-gelu-swish/history.png
1482 1937
error model/Conv1D_LSTM/0.001-nadam-tanh-gelu/history.png
1483 1937
error model/Conv1D_LSTM/0.1-nadam-leaky_relu-leaky_relu/history.png
1484 1937
error model/Conv1D_LSTM/0.01-adam-swish-gelu/history.png
1485 1937
error model/Conv1D_LSTM/0.01-adam-tanh-gelu/history.png
1486 1937
error model/Conv1D_LSTM/0.001-adam-swish-gelu/history.png
1487 1937
error model/Conv1D_LSTM/0-sgd-tanh-leaky_relu/history.png
1488 1937
error model/Conv1D_LSTM/0.001-nadam-leaky_relu-leaky_relu/history.png
1489 1937
error model/Conv1D_LSTM/0.0001-sgd-swish-relu/history.png
1490 1937
error model/Conv1D_LSTM/0.001-nadam-swish-gelu/history.png
1491 1937
error model/Conv1D_LSTM/0-adam-tanh-swish/history.png
1492 1937
error model/Conv1D_LSTM/0-sgd-gelu-swish/history.png
1493 1937
error model/Conv1D_LSTM/0.001-sgd-leaky_relu-relu/history.png
1494 1937
error model/Conv1D_LSTM/0.1-adam-leaky_relu-gelu/history.png
1495 1937
error model/Conv1D_LSTM/0.0001

error model/Conv1D_LSTM/0-adam-gelu-relu/history.png
1601 1937
error model/Conv1D_LSTM/0.1-adam-tanh-swish/history.png
1602 1937
error model/Conv1D_LSTM/0.1-nadam-tanh-leaky_relu/history.png
1603 1937
error model/Conv1D_LSTM/0.01-nadam-gelu-gelu/history.png
1604 1937
error model/Conv1D_LSTM/0.01-adam-leaky_relu-swish/history.png
1605 1937
error model/Conv1D_LSTM/0-nadam-swish-gelu/history.png
1606 1937
error model/Conv1D_LSTM/0.0001-nadam-leaky_relu-gelu/history.png
1607 1937
error model/Conv1D_LSTM/0.01-adam-leaky_relu-gelu/history.png
1608 1937
error model/Conv1D_LSTM/0.1-adam-leaky_relu-swish/history.png
1609 1937
error model/Conv1D_LSTM/0-adam-swish-gelu/history.png
1610 1937
error model/Conv1D_LSTM/0-sgd-tanh-gelu/history.png
1611 1937
error model/Conv1D_LSTM/0.1-adam-swish-swish/history.png
1612 1937
error model/Conv1D_LSTM/0.0001-adam-gelu-gelu/history.png
1613 1937
error model/Conv1D_LSTM/0.1-adam-gelu-relu/history.png
1614 1937
error model/Conv1D_LSTM/0.001-nadam-swish-swish/h

error model/Conv2D_1/0-nadam-tanh-leaky_relu/history.png
1724 1937
error model/Conv2D_1/0.0001-sgd-gelu-relu/history.png
1725 1937
error model/Conv2D_1/0.01-sgd-leaky_relu-gelu/history.png
1726 1937
error model/Conv2D_1/0.01-adam-relu-gelu/history.png
1727 1937
error model/Conv2D_1/0-nadam-relu-gelu/history.png
1728 1937
error model/Conv2D_1/0-sgd-relu-leaky_relu/history.png
1729 1937
error model/Conv2D_1/0.01-adam-relu-leaky_relu/history.png
1730 1937
error model/Conv2D_1/0.001-adam-gelu-leaky_relu/history.png
1731 1937
error model/Conv2D_1/0.001-nadam-gelu-swish/history.png
1732 1937
error model/Conv2D_1/0-nadam-gelu-swish/history.png
1733 1937
error model/Conv2D_1/0.001-nadam-leaky_relu-relu/history.png
1734 1937
error model/Conv2D_1/0-sgd-relu-gelu/history.png
1735 1937
error model/Conv2D_1/0.01-nadam-tanh-relu/history.png
1736 1937
error model/Conv2D_1/0.01-nadam-tanh-swish/history.png
1737 1937
error model/Conv2D_1/0.1-sgd-leaky_relu-leaky_relu/history.png
1738 1937
error model/C

error model/Conv2D_1/0.1-nadam-leaky_relu-swish/history.png
1849 1937
error model/Conv2D_1/0.0001-adam-tanh-gelu/history.png
1850 1937
error model/Conv2D_1/0-sgd-gelu-leaky_relu/history.png
1851 1937
error model/Conv2D_1/0.001-sgd-relu-swish/history.png
1852 1937
error model/Conv2D_1/0.01-sgd-tanh-leaky_relu/history.png
1853 1937
error model/Conv2D_1/0.001-adam-leaky_relu-swish/history.png
1854 1937
error model/Conv2D_1/0-adam-gelu-swish/history.png
1855 1937
error model/Conv2D_1/0.01-adam-swish-swish/history.png
1856 1937
error model/Conv2D_1/0.1-sgd-tanh-gelu/history.png
1857 1937
error model/Conv2D_1/0-nadam-gelu-relu/history.png
1858 1937
error model/Conv2D_1/0.1-sgd-tanh-swish/history.png
1859 1937
error model/Conv2D_1/0.0001-nadam-leaky_relu-swish/history.png
1860 1937
error model/Conv2D_1/0.001-adam-tanh-leaky_relu/history.png
1861 1937
error model/Conv2D_1/0.0001-adam-tanh-swish/history.png
1862 1937
error model/Conv2D_1/0.0001-adam-gelu-leaky_relu/history.png
1863 1937
error m

In [6]:
exist_list = pd.concat([pd.read_csv("inference_groupby_veg_regression.csv"),
                        pd.read_csv("inference_groupby_veg_regression_2.csv"),
                        pd.read_csv("inference_groupby_veg_regression_3.csv"),
                        pd.read_csv("inference_groupby_veg_regression_4.csv"),
                        pd.read_csv("inference_groupby_veg_regression_5.csv"),
                        pd.read_csv("inference_groupby_veg_regression_6.csv")
                       ]).reset_index(drop = True)
list_route = [os.path.join(path, name) for path, subdirs, files in os.walk('model') for name in files]
list_route = [x for x in list_route if 'history.png' in x]
list_route = list(set(list_route)-set(['/'.join(x.split("/")[:-1])+'/history.png' for x in pd.unique(exist_list['source'])]))
print(len(list_route))
data = cal_all_acc(list_route)
data[0].to_csv("inference_all_veg_regression_7.csv", index = False)
data[1].to_csv("inference_groupby_veg_regression_7.csv", index = False)
pd.DataFrame(sum(data[2],[])).to_csv("inference_mse_veg_regression_7.csv", index = False)

0


ValueError: No objects to concatenate

In [11]:
df = pd.concat([pd.read_csv("inference_groupby_veg_regression.csv"),
                pd.read_csv("inference_groupby_veg_regression_2.csv"),
                pd.read_csv("inference_groupby_veg_regression_3.csv"),
                pd.read_csv("inference_groupby_veg_regression_4.csv"),
                pd.read_csv("inference_groupby_veg_regression_5.csv"),
                pd.read_csv("inference_groupby_veg_regression_6.csv")
                       ]).reset_index(drop = True)
df.to_csv("regression.csv", index = False)

In [12]:
mse = pd.concat([pd.read_csv("inference_mse_veg_regression.csv"),
                pd.read_csv("inference_mse_veg_regression_2.csv"),
                pd.read_csv("inference_mse_veg_regression_3.csv"),
                pd.read_csv("inference_mse_veg_regression_4.csv"),
                pd.read_csv("inference_mse_veg_regression_5.csv"),
                pd.read_csv("inference_mse_veg_regression_6.csv")
                       ]).reset_index(drop = True)
mse.to_csv("mse.csv", index = False)

In [9]:
df[df['total-per']>0.7].groupby('source').size().rename('num').sort_values()

source
model/Conv1D_LSTM/0-sgd-relu-relu/weights_mse.hdf5                  1
model/Conv1D_LSTM/0.0001-sgd-leaky_relu-swish/weights_loss.hdf5     1
model/Conv1D_LSTM/0.0001-sgd-leaky_relu-swish/weights_mse.hdf5      1
model/Conv1D_LSTM/0.0001-sgd-relu-gelu/final.h5                     1
model/Conv1D_LSTM/0.0001-sgd-relu-gelu/weights_loss.hdf5            1
                                                                   ..
model/Conv1D_LSTM/0.0001-sgd-gelu-relu/final.h5                     9
model/Conv1D_LSTM/0.0001-sgd-gelu-relu/weights_mse.hdf5             9
model/Conv1D_LSTM/0.0001-nadam-relu-relu/weights_mse.hdf5          10
model/Conv1D_LSTM/0.0001-nadam-relu-relu/weights_loss.hdf5         10
model/Conv1D_LSTM/0.0001-nadam-relu-relu/final.h5                  10
Name: num, Length: 1451, dtype: int64

In [10]:
df[(df['source']=='model/Conv1D_LSTM/0.0001-nadam-relu-relu/weights_loss.hdf5')]
#{'高於成本價格': 0, '低於成本價格': 1, '上漲': 0, '持平':1, '下跌':2}

,var,0-False,1-True,2-False,2-True,0-True,1-False,0-per,1-per,2-per,total-per,source,dataset
48005,漲跌狀態-02,79.0,773.0,251.0,1068.0,977.0,747.0,0.925189,0.508553,0.809704,0.723492,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48006,漲跌狀態-03,96.0,1048.0,303.0,1017.0,961.0,470.0,0.909177,0.690382,0.770455,0.776893,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48007,漲跌狀態-04,80.0,1055.0,283.0,1040.0,960.0,477.0,0.923077,0.688642,0.786092,0.784339,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48008,漲跌狀態-05,75.0,1075.0,189.0,1145.0,982.0,429.0,0.929044,0.714761,0.858321,0.822080,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48009,漲跌狀態-06,106.0,1172.0,209.0,1116.0,924.0,368.0,0.897087,0.761039,0.842264,0.824647,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48020,漲跌狀態-02,15.0,139.0,55.0,182.0,179.0,118.0,0.922680,0.540856,0.767932,0.726744,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48021,漲跌狀態-03,17.0,161.0,67.0,169.0,176.0,98.0,0.911917,0.621622,0.716102,0.735465,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48022,漲跌狀態-04,19.0,161.0,59.0,174.0,192.0,83.0,0.909953,0.659836,0.746781,0.765988,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48023,漲跌狀態-05,20.0,171.0,34.0,188.0,174.0,101.0,0.896907,0.628676,0.846847,0.774709,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48024,漲跌狀態-06,59.0,157.0,38.0,193.0,162.0,79.0,0.733032,0.665254,0.835498,0.744186,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val


,var,0-False,1-True,2-False,2-True,0-True,1-False,0-per,1-per,2-per,total-per,source,dataset
48005,漲跌狀態-02,79.0,773.0,251.0,1068.0,977.0,747.0,0.925189,0.508553,0.809704,0.723492,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48006,漲跌狀態-03,96.0,1048.0,303.0,1017.0,961.0,470.0,0.909177,0.690382,0.770455,0.776893,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48007,漲跌狀態-04,80.0,1055.0,283.0,1040.0,960.0,477.0,0.923077,0.688642,0.786092,0.784339,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48008,漲跌狀態-05,75.0,1075.0,189.0,1145.0,982.0,429.0,0.929044,0.714761,0.858321,0.822080,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48009,漲跌狀態-06,106.0,1172.0,209.0,1116.0,924.0,368.0,0.897087,0.761039,0.842264,0.824647,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,train
48020,漲跌狀態-02,15.0,139.0,55.0,182.0,179.0,118.0,0.922680,0.540856,0.767932,0.726744,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48021,漲跌狀態-03,17.0,161.0,67.0,169.0,176.0,98.0,0.911917,0.621622,0.716102,0.735465,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48022,漲跌狀態-04,19.0,161.0,59.0,174.0,192.0,83.0,0.909953,0.659836,0.746781,0.765988,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48023,漲跌狀態-05,20.0,171.0,34.0,188.0,174.0,101.0,0.896907,0.628676,0.846847,0.774709,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val
48024,漲跌狀態-06,59.0,157.0,38.0,193.0,162.0,79.0,0.733032,0.665254,0.835498,0.744186,model/Conv1D_LSTM/0.0001-nadam-relu-relu/weigh...,val


In [19]:
banana_X_train.to_excel("veg_X_train.xlsx",index = False)
banana_X_val.to_excel("veg_X_val.xlsx",index = False)
banana_Y_train.to_excel("veg_Y_train.xlsx",index = False)
banana_Y_val.to_excel("veg_Y_val.xlsx",index = False)